In [1]:
import pandas as pd
import numpy as np
from os import path
from CSVUtils import *
import ta
import matplotlib.pyplot as plt
import seaborn as sn
import calendar
from pprint import pprint

In [2]:
DIR = "./from github/Stock-Trading-Environment/data"
nameList = ["^BVSP", "^TWII", "^IXIC"]
df_list = []
startDate = pd.to_datetime("2000-01-01")
endDate = pd.to_datetime("2020-01-31")

for name in nameList:
    df = csv2df(csv_path=DIR, csv_name=name+".csv", source="yahoo")
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.dropna()
    df_list.append(df)

In [3]:
twd_df = pd.read_csv("./input/currency/TWD_1995-2020.csv")
brl_df = pd.read_csv("./input/currency/BRL_1995-2020.csv")

twd_df['Date'] = pd.to_datetime(twd_df['Date'])
brl_df['Date'] = pd.to_datetime(brl_df['Date'])

In [4]:
twd_change = twd_df[['Date','Change']]
brl_change = brl_df[['Date','Change']]

In [5]:
def stem_df(df, intersect_dates, startDate, endDate):
    df.dropna(inplace = True)
    df = df[df['Date'].isin(intersect_dates)].reset_index(drop=True)
    df = df[(df['Date'] >= startDate) & (df['Date'] <= endDate)]
    df = df.reset_index(drop=True)
    return df

In [6]:
intersect_dates = df_list[0]['Date']
for df in df_list[1:]+[twd_change,brl_change]:
    df.dropna(inplace = True)
    intersect_dates = np.intersect1d(intersect_dates, df['Date'])

C:\Users\lzcai\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
for i, df in enumerate(df_list):
    df_list[i] = stem_df(df, intersect_dates, startDate, endDate)
twd_change = stem_df(twd_change, intersect_dates, startDate, endDate)
brl_change = stem_df(brl_change, intersect_dates, startDate, endDate)

C:\Users\lzcai\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
df_list[0]['FX Change'] = brl_change['Change']
df_list[1]['FX Change'] = twd_change['Change']
df_list[2]['FX Change'] = 0

In [9]:
for i, df in enumerate(df_list):
    df['Cum FX Change'] = (df['FX Change']+1).cumprod()
    df['Actual Price'] = df['Price']*df['Cum FX Change']

In [10]:
df_list[0].head()

,Date,Open,High,Low,Price,Vol,Change,FX Change,Cum FX Change,Actual Price
0,2000-01-04,16908.0,16908.0,15851.0,15851.0,0.0,-0.063733,-0.0157,0.984300,15602.139300
1,2000-01-05,15871.0,16302.0,15350.0,16245.0,0.0,0.024856,0.0061,0.990304,16087.492216
2,2000-01-06,16237.0,16499.0,15977.0,16107.0,0.0,-0.008495,-0.0017,0.988621,15923.713821
3,2000-01-07,16125.0,16449.0,16125.0,16309.0,0.0,0.012541,0.0074,0.995937,16242.728478
4,2000-01-10,16325.0,17057.0,16325.0,17022.0,0.0,0.043718,0.0075,1.003406,17079.977441


In [11]:
df_list[0].to_csv("./from github\\Stock-Trading-Environment\\data\\^BVSP_new.csv", index=False)
df_list[1].to_csv("./from github\\Stock-Trading-Environment\\data\\^TWII_new.csv", index=False)
df_list[2].to_csv("./from github\\Stock-Trading-Environment\\data\\^IXIC_new.csv", index=False)

In [20]:
# 0316
df = df_list[2]
price_label = "Actual Price"

df = df.sort_values('Date').reset_index(drop=True)
# Add TA Indicators
# <1> EMA
df['EMA'] = df[price_label].ewm(span=15).mean()
# <2> MACD_diff
df['MACD_diff'] = ta.trend.macd_diff(df[price_label], fillna=True)
macd_direction = df['MACD_diff']/np.abs(df['MACD_diff']) # 1: No change, -1: Change Sign
# <3> MACD_change
df['MACD_change'] = (-1*macd_direction*macd_direction.shift(1)+1)/2 # 1: Change Sign, 0: No Change

# <4> delta_time: How many days since the last trend change
delta_time = [] 
for i in df['MACD_change']:
    if len(delta_time) == 0:
        result = 0
    elif i==0:
        result = delta_time[-1]+1
    else: #Nan or 1
        result = 0
    delta_time.append(result)
df['delta_time'] = delta_time

# <5> RSI
df['RSI'] = ta.momentum.RSIIndicator(df[price_label], fillna=True).rsi() 
df['RSI'] /= df['RSI'][0]
    

In [21]:
df

,Date,Open,High,Low,Price,Vol,Change,FX Change,Cum FX Change,Actual Price,EMA,MACD_diff,MACD_change,delta_time,RSI
0,2000-01-04,4020.000000,4073.250000,3898.229980,3901.689941,1511840000,-0.055544,0,1,3901.689941,3901.689941,0.000000,NaN,0,1.000000
1,2000-01-05,3854.350098,3924.209961,3734.870117,3877.540039,1735670000,-0.006190,0,1,3877.540039,3888.809993,-1.541190,NaN,0,0.000000
2,2000-01-06,3834.439941,3868.760010,3715.620117,3727.129883,1598320000,-0.038790,0,1,3727.129883,3827.582023,-12.021699,0.0,1,0.000000
3,2000-01-07,3711.090088,3882.669922,3711.090088,3882.620117,1634930000,0.041718,0,1,3882.620117,3844.207099,-8.006545,0.0,2,0.492089
4,2000-01-10,4002.229980,4072.360107,3958.830078,4049.669922,1691710000,0.043025,0,1,4049.669922,3896.934104,5.639681,1.0,0,0.676354
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4597,2020-01-15,9253.759766,9298.820313,9231.139648,9258.700195,2435650000,0.000797,0,1,9258.700195,9093.150542,10.393574,0.0,18,0.746215
4598,2020-01-16,9313.450195,9357.919922,9301.320313,9357.129883,2301400000,0.010631,0,1,9357.129883,9126.147959,13.224174,0.0,19,0.781619
4599,2020-01-17,9392.370117,9393.480469,9346.809570,9388.940430,2522670000,0.003400,0,1,9388.940430,9158.997018,14.932877,0.0,20,0.791731
4600,2020-01-30,9211.150391,9303.000000,9185.179688,9298.929688,2333500000,0.002563,0,1,9298.929688,9176.488602,8.040441,0.0,21,0.693830
